## Transmitter

In [1]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys
import copy

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate
import threading

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray
from  scipy.io.wavfile import read as wavread
import newax25 as ax25

import multiprocessing

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign
from queue import *

import zlib
from PIL import Image
import scipy

import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

from trans_utils import *
from proc_funs import *

%matplotlib inline

In [2]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 USB PnP Sound Device: Audio (hw:1,0), ALSA (1 in, 2 out)
  3 sysdefault, ALSA (0 in, 128 out)
  4 dmix, ALSA (0 in, 2 out)
* 5 default, ALSA (1 in, 2 out)

In [3]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1
#cool stuff

In [4]:
# def queueREPLAY_callback(indata,outdata, frames, time, status):
#     if status:
#         print(status)
#     outdata[:] = indata
#     Qin.put( indata.copy()[:,0] )  # Global queue

## Make Modem

Set up modem arguments

In [5]:
Abuffer = 512
Nchunks = 1
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 1)

Set up GPIO pins

In [6]:
GPIO.setmode(GPIO.BOARD)
PTT = 12
GPIO.setup(PTT, GPIO.OUT, initial = 0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  This is separate from the ipykernel package so we can avoid doing imports until


## Transmitter Setup

In [7]:
callsign = "KM6QGZ"
img_f_name = 'createrLake.tiff'
trans_scalar = 0.2
comp_val = 9
num_preflags = 80
num_postflags = 80
for_proc_fun, rev_proc_fun = down_up_samp(12, 18, (432, 1200))

## Process Image to Bytes and Compress

In [8]:
bytes_to_comp, shapes = for_proc_fun(img_f_name)
bytes_comp = zlib.compress(bytes_to_comp, comp_val)
print('Compressed by: ', comp_rate(bytes_to_comp, bytes_comp))

Compressed by:  0.010503040353786623


In [9]:
print(len(bytes_comp))

7160


In [10]:
shapes

[(36, 67), (36, 67), (36, 67)]

## Make and Modulate Packets

In [11]:
negative_nancy = [shapes[0][0], shapes[0][1], shapes[1][0], shapes[1][1], shapes[2][0], shapes[2][1]]
str_nancy = 

SyntaxError: invalid syntax (<ipython-input-11-bf2810af409f>, line 2)

In [12]:
print("Putting packets in Queue")

# For you to complete
Qout = Queue()
dest_int = 1
block_size = 256

for start_ind in range(0,len(bytes_comp), block_size):
    curr_block = bytes_comp[start_ind:min(start_ind+block_size, len(bytes_comp))]
    Qout.put(modem.modulatPacketAX25('', b'', str(dest_int), curr_block, preflags=num_preflags, postflags=num_postflags))
    dest_int += 1
print("Done")
print(str(Qout.qsize()) + ' total packets.')

Putting packets in Queue
Done
28 total packets.


## Transmit Packets

In [13]:
starttime = time.time()

GPIO.output(PTT, GPIO.HIGH)
time.sleep(0.25)
num_pks = Qout.qsize()
for n in range(Qout.qsize()):
    curr_packet = Qout.get()
    print('Packet '+str(n+1)+'/'+str(num_pks))
    sd.play(curr_packet*trans_scalar,device=usb_idx,blocking=True)
GPIO.output(PTT, GPIO.LOW)

print("Done")
print(time.time() - starttime)

Packet 1/28
Packet 2/28
Packet 3/28
Packet 4/28
Packet 5/28
Packet 6/28
Packet 7/28
Packet 8/28
Packet 9/28
Packet 10/28
Packet 11/28
Packet 12/28
Packet 13/28
Packet 14/28
Packet 15/28
Packet 16/28
Packet 17/28
Packet 18/28
Packet 19/28
Packet 20/28
Packet 21/28
Packet 22/28
Packet 23/28
Packet 24/28
Packet 25/28
Packet 26/28
Packet 27/28
Packet 28/28
Done
42.71046495437622
